In [6]:
%matplotlib inline
import numpy as np
import random
import torch
from torch.utils import data
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 线性回归的简洁实现

## 准备批量化数据

In [98]:
def synthetic_data(w, b, num_examples):  
    """生成y=Xw+b+噪声"""
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape) # y.shape: torch.Size([1000,1])
    return X, y.reshape(-1,1) # y.shape: torch.Size([1000, 1])
    # return 为什么y要进行reshape？

In [99]:
true_w = torch.tensor([2, -3.4])  # 给定真实的权重
true_b = 4.2                      # 给定真实的偏置
features, labels = synthetic_data(true_w, true_b, 1000)   # 生成特征矩阵与标签向量

- 调用torch框架中现有的API来读取数据，并构造数据迭代器

In [64]:
def load_array(data_arrays, batch_size, is_train=True):  
    """构造一个PyTorch数据迭代器
    is_train：是否在每个迭代周期打乱数据排序
    """
    dataset = data.TensorDataset(*data_arrays) # data_arrays: (features, labels)全数据样本
    return data.DataLoader(dataset, batch_size, shuffle=is_train) # torch.utils.data.dataloader.DataLoader

- ```python
from torch.utils import data
data.DataLoader(dataset, batch_size=1, shuffle=False)
```

    - `dataset`：需要加载的数据集
    - `batch_size`：一次加载的样本数量（批量），是`int`类型
    - `shuffle`：每代（epoch）训练是否随机样本排序，是`bool`类型，降低对数据的过拟合
    - 返回一个可迭代对象`iterable`

- ```python
data.TensorDataset(*tensors)
```
    - `tensors`：一系列张量，这些张量的第一维度拥有相同的大小
    - 类似于`Python`的`zip`函数

In [7]:
a = torch.tensor([[1,2,4],[4,5,6],[7,8,9]])
b = torch.tensor([10,11,12])
c = data.TensorDataset(a,b)
c[0]

(tensor([1, 2, 4]), tensor(10))

In [100]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)  # （features,labels）构成一个tuple作为load_array函数的第一个参数

>【注意】data_iter并不是一个真的iterator, 要想当iterator来用，方法是：

In [101]:
my_iter = iter(data_iter)  # Python内置的迭代器构造函数iter()
next(my_iter)

[tensor([[ 1.2509, -0.6020],
         [ 1.0021,  0.4213],
         [-0.1978, -0.1003],
         [-0.0545, -0.6706],
         [-1.4072,  0.7688],
         [ 1.6437,  0.6058],
         [-1.8432, -1.5889],
         [ 0.0451,  0.8303],
         [-0.2651, -0.1089],
         [ 0.2316, -1.2513]]),
 tensor([[ 8.7474],
         [ 4.7749],
         [ 4.1285],
         [ 6.3988],
         [-1.2307],
         [ 5.4103],
         [ 5.9093],
         [ 1.4725],
         [ 4.0492],
         [ 8.9318]])]

## 提出拟合数据的模型

### 容器（containers）

- 容器：用`pytorch`构建网络图的基本组件

- 容器的子类：
    - `Module`：所有神经网络的基类
    - `Sequential`：顺序构成的容器组合
    - 。。。

- `Module`：通过继承该类建立自己的模型
- `Module`可以包含其他`Module`，即允许嵌套

In [ ]:
# 简单线性模型
import torch.nn as nn


class Linear_Net(nn.Module):
    def __init__(self, n_features, n_out):
        # 参数n_features：输入预测属性的个数，n_out：预测值的个数
        super().__init__()
        # 构建网络结构
        self.predict = nn.Linear(n_features, n_out)

    def forward(self, x):
        # 前向计算，x是输入的样本
        x = self.predict(x)
        return x

- ```python
torch.nn.Linear(in_features,out_features,bias=True)
```
    - `in_features`：输入样本的维度，`int`类型
    - `out_features`：输出样本的维度，`int`类型
    - `bias`：是否学习偏置参数，`bool`类型

- 对输入数据$\mathbf{X}$进行线性变换

$$
\boldsymbol{y} = \boldsymbol{w}\mathbf{X}+b
$$

- $\mathbf{X}$的形状为 (*, in_features)
- $\boldsymbol{y}$的形状为（*，out_features)
- *代表数据包含的样本数量

- 权重$\boldsymbol{w}$的形状为(out_features,in_features)，权重初始值从均匀分布$U(-\sqrt{k},\sqrt{k})$中随机产生，其中$k=\tfrac{1}{\text {in_features}}$

- 偏置$b$的形状为(out_features)，偏置的初始值也从均匀分布$U(-\sqrt{k},\sqrt{k})$中随机产生，其中$k=\tfrac{1}{\text{in_features}}$

- `Sequential`将多个`Module`**串联**在一起
    - 包含的`Module`的顺序与构建该`Sequential`时候输入的`Module`顺序一致

<center><img src="../img/3_linear_network/sequential.png" width=80%></center>

- `Sequential`的`forward()`方法将接收的输入张量传递给第一个`Module`，
- 然后将其计算结果作为第二个`Module`的输入，
- 依次向后续`Module`传递

### 使用框架预定义的层构建模型

- 定义模型变量`net`，是`Sequential`类的一个实例

- 在`PyTorch`中，通过`Linear`类定义全连接层
- 向`Linear`传递两个参数
    - 输入特征形状，即2
    - 输出特征形状，输出特征形状为单个标量，因此为1

In [66]:
from torch import nn    ## nn是neural network

net = nn.Sequential(nn.Linear(2, 1))

In [67]:
net

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)

### 初始化模型参数

- 深度学习框架通常用预定义的方法初始化参数

- 初始化参数的方法
    - 用`net[0]`选择网络的第一图层，如果是多层网络，用序号选择相应的图层
    - `weight.data`访问权重参数，`bias.data`访问偏置参数
    - `normal_()`和`fill_()`重新填写参数

In [102]:
print(f'net网络的权重初始化从均值为0、标准差为0.01的正态分布随机采样，\
      \n{net[0].weight.data.normal_(0, 0.01)}')
print(f'net网络的偏置初始化为0，\n{net[0].bias.data.fill_(0)}')

net网络的权重初始化从均值为0、标准差为0.01的正态分布随机采样，      
tensor([[-0.0032, -0.0032]])
net网络的偏置初始化为0，
tensor([0.])


- 查看网络的参数

- ```python
net.parameters()
```

    - `net`：建立的网络模型

In [51]:
f'网络参数的类型为{type(net.parameters())}'
paras = net.parameters()  # 实例化对象
next(paras,'end')  # next()有一个默认参数，当迭代对象完毕的时候输出该默认参数，否则会报错
next(paras,'end')
next(paras,'end')

"网络参数的类型为<class 'generator'>"

Parameter containing:
tensor([[-0.0208,  0.0055]], requires_grad=True)

Parameter containing:
tensor([0.], requires_grad=True)

'end'

## 定义损失函数

- 计算均方误差使用的是`torch.nn`的`MSELoss`类，也称为平方$L_2$范数
    - 返回所有样本损失的平均值

In [103]:
loss = nn.MSELoss() # MeanSquaredError

## 定义优化算法

- 小批量随机梯度下降算法（stochastic gradient descent, SGD）是一种优化神经网络的标准工具实例
- `PyTorch`在`optim`模块中实现了该算法的许多变种

- 使用优化算法的方法：


1. 需要实例化一个`SGD`对象
1. 指定参数
    - 需要优化的参数，可以通过`net.parameters()`获得
    - 算法的超参，例如学习速率`lr`

In [104]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

3. 进行单次优化
    - 一旦梯度经过backward()计算后，调用`step()`函数进行优化
```python
trainer.step()
```

## 正式开始训练

- 每个迭代周期将完整遍历一次数据集（`train_data`），不停地从中获取一个小批量的输入和相应的标签

- 对于每一个小批量，我们会进行以下步骤:
    * 通过调用`net(X)`生成预测并计算损失$l$（前向传播）
    * 通过进行反向传播来计算梯度
    * 通过调用优化器来更新模型参数

In [105]:
num_epochs = 10    # 10代训练
for epoch in range(num_epochs):
    for X, y in data_iter:  # 遍历所有数据
        trainer.zero_grad() # 梯度置0
        l = loss(net(X) ,y) # 正向计算损失，损失函数的参数为预测值和真值
        l.backward()        # 反向传播计算梯度
        trainer.step()      # 用SGD算法更新参数

    l = loss(net(features), labels) # 计算每代训练的损失，注意用全部数据
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000247
epoch 2, loss 0.000104
epoch 3, loss 0.000104
epoch 4, loss 0.000104
epoch 5, loss 0.000105
epoch 6, loss 0.000104
epoch 7, loss 0.000104
epoch 8, loss 0.000105
epoch 9, loss 0.000105
epoch 10, loss 0.000104


- 比较生成数据集的真实参数和通过有限数据训练获得的模型参数

In [106]:
w = net[0].weight.data
print(f'w的估计误差：, {true_w - w.reshape(true_w.shape)}')
b = net[0].bias.data
print(f'b的估计误差：, {true_b - b}')

w的估计误差：, tensor([0.0003, 0.0004])
b的估计误差：, tensor([4.9114e-05])
